In [2]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import timm
import torch

# sns.set_theme()

%matplotlib widget
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

torch.manual_seed(42)

Need a model that takes a sequence of images as input and chooses two among them for the best exposure fusion result

In [4]:
from math import comb

import pandas as pd
import torchvision as tv
from torch.utils.data import DataLoader, Subset, random_split
from torchvision import transforms

from dhdrnet.dataset import CachingDataset
from dhdrnet.fc_train import get_loaders
from dhdrnet.free_choice_model import FreeChoiceDHDRNet

exposure_vals = [-5, -3, -1, 0, 1, 3, 5]

train_file = Path("../../precomputed_data/train_current.csv")
train_image_names = pd.read_csv(train_file, squeeze=True, dtype=str).to_list()

ds = CachingDataset(
    data_dir=Path("../../data"),
    image_names=train_image_names,
    exposure_values=exposure_vals,
    transform=transforms.Compose(
        [transforms.ToPILImage(), transforms.Resize((300, 300)), transforms.ToTensor()]
    ),
)
tdl, vdl = get_loaders(ds, 4, trainval_split=0.8)

In [4]:
for batch in dl:
    images, ev_classes, scores = batch
    print(f"{images.shape=}")
    print(f"{ev_classes}")
    break

NameError: name 'dl' is not defined

datasets are being processed by the dataloader correctly :)

In [ ]:
train_ds = Subset(ds, range(20))
val_ds = Subset(ds, range(20, 40))

In [ ]:
tf = Path("../../precomputed_data/test_current.csv")
pd.read_csv(tf, squeeze=True)

In [ ]:
parts = (f.stem.split("_")[-1] for f in Path("../../logs/").glob("exp_*"))
max(int(p) for p in parts if p.isdigit())

In [6]:
from dhdrnet.util import evpairs_to_classes, get_valid_exp_path

classmapping = evpairs_to_classes(exposure_vals)
get_valid_exp_path(Path("../../logs"))

PosixPath('../../logs/exp_14')

In [7]:
df = pd.read_json("../../data/store/best_evs.json", lines=True, orient="records")
df = df.drop("exposure_values", axis="columns")
df["evs"] = df["evs"].apply(tuple)
df["ev_class"] = df["evs"].apply(lambda x: classmapping[x])
df.hist()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([[<AxesSubplot:title={'center':'score'}>,
        <AxesSubplot:title={'center':'ev_class'}>]], dtype=object)

In [39]:
classmapping

{(-5, -3): 0,
 (-5, -1): 1,
 (-5, 0): 2,
 (-5, 1): 3,
 (-5, 3): 4,
 (-5, 5): 5,
 (-3, -1): 6,
 (-3, 0): 7,
 (-3, 1): 8,
 (-3, 3): 9,
 (-3, 5): 10,
 (-1, 0): 11,
 (-1, 1): 12,
 (-1, 3): 13,
 (-1, 5): 14,
 (0, 1): 15,
 (0, 3): 16,
 (0, 5): 17,
 (1, 3): 18,
 (1, 5): 19,
 (3, 5): 20}

In [8]:
df.describe()

,score,ev_class
count,4000.000000,4000.000000
mean,25.678320,12.651250
std,1.923986,4.727442
min,16.755633,1.000000
25%,24.451106,12.000000
50%,25.718093,13.000000
75%,26.871488,16.000000
max,35.663063,20.000000


In [12]:
df.groupby("ev_class").mean().plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='ev_class'>